In [10]:
import requests
import json
import fire
import pandas as pd
import numpy as np
import pymysql

In [11]:
URL = "https://api.quash.ai"
DATA = "./data/datos.json"
ENDPOINT = "/scoring"
APIKEY = "Ku9sm6Nr6XAEXlWfLNyPmDhkTvTa4EnS"

In [12]:
def setcut_prob(p, grupo_ = True):
    p = p*100
    cuts = np.array(sorted([100, 81.09, 72.06, 63.19, 54.27, 45.11, 36.10, 27.47, 18.17, 9.05, 0.00]))
    cosechas = np.array(sorted([1.63, 1.84, 1.89, 3.19, 3.76, 5.77, 7.48, 13.17, 15.20, 17.35], reverse = False))
                                 
    
    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))
    
    for cut in cuts:
        if cut > p:
            break
            
    grupo = np.where(cuts == cut)[0][0] 
    if grupo_:
        return grupo
    else:
        return cosechas[grupo - 1]
    
def setcut_prob_quash(p, grupo_ = True):
    p = p*100
    cuts = np.array(sorted([100, 96.8, 93.6, 84.1, 75.4, 50.2, 0.00]))
    cosechas = np.array(sorted([1, 1.98, 4.06, 6.11, 7.65, 11.8, 14.76], reverse = True))
                                 
    
    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))
    
    for cut in cuts:
        if cut > p:
            break
            
    grupo = np.where(cuts == cut)[0][0] 
    if grupo_:
        return grupo
    else:
        return cosechas[grupo - 1]

In [13]:
setcut_prob_quash(0.6, grupo_ = False)

11.8

**Cargando los datos**

In [14]:
data_resultado_quilum = pd.read_excel("./data/240221 ANALISIS DE MODELOS.xlsx", sheet_name = "Hoja1")
data_resultado_quash = pd.read_excel("./data/240221 ANALISIS DE MODELOS.xlsx", sheet_name = "Hoja1")
data_modelo = pd.read_excel("./data/240221 ANALISIS DE MODELOS.xlsx", sheet_name = "Hoja2", header=1)

In [15]:
data_prospectos_original = pd.read_csv("./data/prospectos_riesgo.csv")

In [16]:
data_prospectos = data_prospectos_original.groupby("id_distribuidor").agg({"monto_autorizado": max, 
                                              "tasa_sugerida":max,
                                              "tasa_minima": min,
                                              "probabilidad" : max,
                                              "grupo_riesgo": max})
data_prospectos = data_prospectos.reset_index()

**Todos los ids en resultado están en los prospectos**

In [17]:
np.setdiff1d(data_resultado_quilum.id_distribuidor.unique(), data_prospectos.id_distribuidor.unique())

array([], dtype=int64)

**Haciendo el merge de la tasa mínima**

In [18]:
data_resultado_quilum = data_resultado_quilum.merge(data_prospectos[['id_distribuidor', 'tasa_minima', 'probabilidad']], on = 'id_distribuidor')

**Añadiendo la pérdida de cosecha esperada**

In [19]:
data_resultado_quilum['perdida_cosecha_esperada_quilum'] = data_resultado_quilum.probabilidad.apply(setcut_prob, grupo_ = False)

In [20]:
data_resultado_quilum

,id_distribuidor,tasa_sugerida,grupo_riesgo,ID ASOCIADO,CAPITAL AUTORIZADO,TASA AUTORIZADA,RECUPERADO,NO RECUPERADO,CERRADO,P COSECHA,tasa_minima,probabilidad,perdida_cosecha_esperada_quilum
0,145442,147,Canasta 1,162807,6000,142,6000.000000,0.000000,SI,0.000000,100,0.0000,1.63
1,145444,147,Canasta 4,162806,6000,142,6000.000000,0.000000,SI,0.000000,110,0.2163,1.89
2,157169,147,Canasta 2,157637,8000,132,8000.000000,0.000000,SI,0.000000,95,0.1268,1.84
3,157171,147,Canasta 4,157634,11000,132,11000.000000,0.000000,SI,0.000000,115,0.2696,1.89
4,157178,147,Canasta 6,157635,11000,132,11000.000000,0.000000,SI,0.000000,127,0.3558,3.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,167070,147,Canasta 4,167255,9000,142,3398.411536,5601.588464,SI,-0.622399,127,0.2365,1.89
1413,167202,0,Canasta 6,167259,9000,142,3398.411536,5601.588464,SI,-0.622399,0,0.3515,3.19
1414,166739,0,Canasta 9,167256,9000,142,3398.411536,5601.588464,SI,-0.622399,0,0.7024,13.17
1415,166784,137,Canasta 4,167257,9000,142,3398.411536,5601.588464,SI,-0.622399,110,0.2319,1.89


## quash

In [21]:
def levenshtein_distance(word1, word2):
    len1, len2 = len(word1), len(word2)
    
    # Crear una matriz para almacenar las distancias parciales
    dp = [[0] * (len2 + 1) for _ in range(len1 + 1)]

    # Inicializar la primera fila y columna
    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j

    # Calcular las distancias parciales
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            cost = 0 if word1[i - 1] == word2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1,      # Eliminación
                           dp[i][j - 1] + 1,      # Inserción
                           dp[i - 1][j - 1] + cost)  # Sustitución

    # El resultado final está en la esquina inferior derecha de la matriz
    return dp[len1][len2]


def MostSimilar(word, list_):
    word = str(word).upper()
    min_distance = np.inf
    index = 0
    for i, element in enumerate(list_):
        d = levenshtein_distance(element, word)
        if d < min_distance:
            min_distance = d
            index = i
            
    return list_[index]

In [22]:
data_modelo

,APPLICANT_ID,INGRESO_MENSUAL,GASTOS_FAMILIARES,OTROS_INGRESOS,INGRESO_NETO,DEPENDIENTES,VENTAS,EDAD,MONTO_MENSUAL_VENTA,COMPRAS,DELEGACION,DELEGACION_NEGOCIO,ESCOLARIDAD,ESTADO,ESTADO_CIVIL,ESTADO_NEGOCIO,SEXO,SUCURSAL,TIPO_NEGOCIO,TIPO_VIVIENDA
0,145442,8000,800.0,2500,"$10,800.00",2,2000,28481,8000,900,Tenango del Aire,Tenango del Aire,Secundaria,México,Soltera (o),México,FEMENINO,AMECAMECA,CALZADO REPARACION Y VENTA,Familiar
1,145444,12200,0.0,1000,"$12,200.00",0,4000,25963,16000,1500,Tenango del Aire,Tenango del Aire,Tecnico,México,Casada (o),México,FEMENINO,AMECAMECA,VENTA DE ALIMENTOS,Familiar
2,157169,16000,1300.0,2100,"$9,600.00",1,4000,32993,16000,1300,Nezahualcóyotl,Nezahualcóyotl,Tecnico,México,Casada (o),México,FEMENINO,NEZAHUALCOYOTL 2,VENTA DE ROPA,Familiar
3,157171,10000,1000.0,1500,"$6,800.00",3,2500,36433,10000,1200,Nezahualcóyotl,Nezahualcóyotl,Bachillerato,México,Unión Libre,México,FEMENINO,NEZAHUALCOYOTL 2,VENTA DE ROPA,Familiar
4,157178,14000,1700.0,2500,"$9,100.00",0,3500,26013,14000,1800,Nezahualcóyotl,Nezahualcóyotl,Secundaria,México,Soltera (o),México,FEMENINO,NEZAHUALCOYOTL 2,JOYERIA/BISUTERIA/FUNDICION,Familiar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,167070,11200,1500.0,2000,"$5,200.00",1,2800,28245,11200,1500,Milpa Alta,Milpa Alta,Secundaria,Distrito Federal,Unión Libre,Distrito Federal,FEMENINO,MILPA ALTA,VENTA DE ROPA,Propia
1413,167202,11200,1400.0,1800,"$4,400.00",1,2800,33246,11200,1500,Milpa Alta,Milpa Alta,Secundaria,Distrito Federal,Unión Libre,Distrito Federal,FEMENINO,MILPA ALTA,VENTA DE ROPA,Familiar
1414,166739,29600,1700.0,2300,"$6,400.00",2,7400,36106,29600,4000,Milpa Alta,Milpa Alta,Secundaria,Distrito Federal,Unión Libre,Distrito Federal,FEMENINO,MILPA ALTA,ANIMALES/VETERINARIA,Propia
1415,166784,9600,1100.0,2400,"$8,000.00",0,2400,24845,9600,1400,Milpa Alta,Milpa Alta,Secundaria,Distrito Federal,Unión Libre,Distrito Federal,FEMENINO,MILPA ALTA,VENTA DE ROPA,Propia


In [23]:
data_modelo.INGRESO_MENSUAL = data_modelo.INGRESO_MENSUAL.astype(int)
data_modelo.GASTOS_FAMILIARES = data_modelo.GASTOS_FAMILIARES.astype(int)
data_modelo.OTROS_INGRESOS = data_modelo.OTROS_INGRESOS.astype(int)
data_modelo.COMPRAS = data_modelo.COMPRAS.astype(int)

In [24]:
data_modelo = data_modelo.loc[data_modelo.MONTO_MENSUAL_VENTA != '4 ANOS']

In [25]:
#data_modelo.MONTO_MENSUAL_VENTA = 
data_modelo.MONTO_MENSUAL_VENTA = data_modelo.MONTO_MENSUAL_VENTA.str.replace("$", "").str.replace("|", "").str.replace(",", "").astype(float).astype(int)

C:\Users\XMF3802_1\AppData\Local\Temp\ipykernel_4204\1086178666.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_modelo.MONTO_MENSUAL_VENTA = data_modelo.MONTO_MENSUAL_VENTA.str.replace("$", "").str.replace("|", "").str.replace(",", "").astype(float).astype(int)
C:\Users\XMF3802_1\AppData\Local\Temp\ipykernel_4204\1086178666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_modelo.MONTO_MENSUAL_VENTA = data_modelo.MONTO_MENSUAL_VENTA.str.replace("$", "").str.replace("|", "").str.replace(",", "").astype(float).astype(int)


In [26]:
ids = ", ".join(data_modelo.APPLICANT_ID.unique().astype(str))

cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                       charset='utf8',db = 'volana')

cursor = cnx.cursor()
sql = f"""
select id as 'APPLICANT_ID', TIMESTAMPDIFF(YEAR, fecha_nacimiento, CURDATE()) AS EDAD from
(select id, fecha_nacimiento from demograficos_distribuidores
union
select id, fecha_nacimiento from cc_prospectos_demograficos) as pen
where pen.id in ({ids})
"""

cursor.execute(sql)
edad = cursor.fetchall()
edad = pd.DataFrame(edad, columns=[i[0] for i in cursor.description])

In [27]:
edad.APPLICANT_ID = edad.APPLICANT_ID.astype(int)
edad.EDAD = edad.EDAD.astype(int)

In [28]:
data_modelo = data_modelo.drop("EDAD", axis = 1).merge(edad, on = 'APPLICANT_ID')

In [29]:
data_modelo = data_modelo[['APPLICANT_ID', 'INGRESO_MENSUAL', 'GASTOS_FAMILIARES',
       'OTROS_INGRESOS', 'INGRESO_NETO', 'DEPENDIENTES', 'VENTAS', 'EDAD',
       'MONTO_MENSUAL_VENTA', 'COMPRAS', 'DELEGACION', 'DELEGACION_NEGOCIO',
       'ESCOLARIDAD', 'ESTADO', 'ESTADO_CIVIL', 'ESTADO_NEGOCIO', 'SEXO',
       'SUCURSAL', 'TIPO_NEGOCIO', 'TIPO_VIVIENDA']]

In [30]:
data_modelo.INGRESO_NETO = data_modelo.INGRESO_NETO.str.replace("$", "").str.replace(",", "").astype(float).astype(int)

C:\Users\XMF3802_1\AppData\Local\Temp\ipykernel_4204\4118494542.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_modelo.INGRESO_NETO = data_modelo.INGRESO_NETO.str.replace("$", "").str.replace(",", "").astype(float).astype(int)


In [31]:
DELGACIONES = [
    "CENTRO", "CHALCO", "CHICOLOAPAN", "CHIMALHUACAN", "IXTAPALUCA", "IZTAPALAPA", "LA_PAZ", "NEZAHUALCOYOTL",
    "VALLE_DE_CHALCO_SOLIDARIDAD", "ACAMBAY", "ACOLMAN", "AMATLAN_DE_LOS_REYES", "AMECAMECA", "ATENCO", "ATLAUTLA",
    "ATOYAC", "ATZACAN", "AYAPANGO", "BENITO_JUAREZ", "CAMERINO_Z._MENDOZA", "CARDENAS", "CHIAUTLA", "CHICONCUAC",
    "CHOCAMAN", "COATZACOALCOS", "COMALCALCO", "CORDOBA", "COSCOMATEPEC", "COSOLEACAQUE", "CUAUHTEMOC", "CUITLAHUAC",
    "CUNDUACAN", "ECATEPEC_DE_MORELOS", "ECATZINGO", "FORTIN", "GUADALAJARA", "GUSTAVO_A._MADERO", "HUATUSCO",
    "HUILOAPAN_DE_CUAUHTEMOC", "HUIMANGUILLO", "HUIXQUILUCAN", "IXHUATLANCILLO", "IXTACZOQUITLAN",
    "IXTLAHUACAN_DE_LOS_MEMBRILLOS", "IZTACALCO", "JUCHITEPEC", "LA_PERLA", "MACUSPANA", "MARIANO_ESCOBEDO",
    "MILPA_ALTA", "MINATITLAN", "NACAJUCA", "NANCHITAL_DE_LAZARO_CARDENAS_DEL_RIO", "NARANJAL", "NOGALES",
    "OMEALCA", "ORIZABA", "OZUMBA", "PAPALOTLA", "RAFAEL_DELGADO", "RIO_BLANCO", "TEMAMATLA", "TENANGO_DEL_AIRE",
    "TEOTIHUACAN", "TEPETLAOXTOC", "TEPETLIXPA", "TEPOTZOTLAN", "TEQUILA", "TEXCOCO", "TEZOYUCA", "TLAHUAC",
    "TLAJOMULCO_DE_ZUNIGA", "TLALMANALCO", "TLAQUEPAQUE", "TLILAPAN", "TOMATLAN", "TONALA", "VENUSTIANO_CARRANZA",
    "XOCHIMILCO", "ZAPOPAN", "ZAPOTLANEJO"
]
ESCOLARIDADES = [
    "BACHILLERATO", "LICENCIATURA", "PRIMARIA", "SECUNDARIA", "TECNICO", "OTRO", "SIN_ESTUDIOS"
]
ESTADOS = [
    "DISTRITO_FEDERAL", "JALISCO", "MEXICO", "TABASCO", "VERACRUZ_DE_IGNACIO_DE_LA_LLAVE", "QUINTANA_ROO"
]
ESTADOS_CIVILES = [
    "CASADO", "SOLTERO", "UNION LIBRE", "VIUDO", "DIVORCIADO", "SEPARADO"
]
SUCURSALES = [
    "AMECAMECA", "CARDENAS", "CHALCO", "CHIMALHUACAN_2", "COATZACOALCOS", "FORTIN", "IXTAPALUCA", "IXTAPALUCA_2",
    "NEZAHUALCOYOTL_2", "ORIZABA", "TEXCOCO", "TONALA", "TONALA_2", "VILLAHERMOSA", "IZTAPALAPA", "MILPA_ALTA",
    "TEXCOCO_2", "TLAHUAC"
]
TIPOS_NEGOCIO = [
    "ABARROTES/MISCELANEAS/PERECEDEROS", "BELLEZA/SALON/ESTETICA/BARBERIA", "CALZADO_REPARACION_Y_VENTA",
    "FABRICA/TALLER/ELABORACION/REPARACION", "FRUTAS/VERDURAS/SEMILLAS/CAMPO/AGROPECUARIAS",
    "JOYERIA/BISUTERIA/FUNDICION", "VENTA_DE_ALIMENTOS", "VENTA_DE_ROPA", "VENTA_POR_CATALOGO", "AGUA/POTABLE/RESIDUAL",
    "AMEDICINA/SALUD/FARMACIA", "ANIMALES/VETERINARIA", "ARTE/ARTESANIAS", "CAFEINTERNET",
    "CARNICERIA/POLLERIA/RASTRO", "CERRAJERIA", "COMBUSTIBLES/GAS/GASOLINA", "DEPORTES/GIMNASIO",
    "DESPERDICIOS/RESIDUOS/RECICLAJE", "DISTRIBUIDOR_CON_EXPERIENCIA", "EDUCACION/ESCUELAS/CENTRO_DE_CAPACITACION",
    "ELECTRICISTA", "EVENTOS/SALONES/SILLAS/MESAS", "FLORES/ARREGLOS", "FOTOGRAFIA/VIDEO", "HERRERIA",
    "IMPRENTA/SERIGRAFIA/ROTULOS", "JARCIERIA", "LAVANDERIA/TINTORERIA/PLANCHADURIA", "MADERA/CARPINTERIA/EBANISTA",
    "MECANICA_AUTOMOTRIZ/INDUSTRIAL", "PANADERIA", "PAPELERIA", "PLOMERIA", "TLAPALERIA/FERRETERIA/MATERIALES_DE_CONSTRUCCION",
    "TORTILLERIA/MOLINO", "VENTA_DE_AUTOS/MOTOCICLETAS/LOTES/AUTOPARTES", "VENTA_DE_CALZADO",
    "VENTA_DE_CELULARES/TECNOLOGIA", "VENTA_DE_COMIDA", "VENTA_DE_MUEBLES/ELECTRONICOS/ELECTRODOMESTICOS"
]
TIPOS_VIVIENDA = [
    "FAMILIAR", "PROPIA", "RENTADA", "OTRO"
]


In [33]:
data_modelo.DELEGACION = data_modelo.DELEGACION.apply(MostSimilar, list_ = DELGACIONES)
data_modelo.DELEGACION_NEGOCIO = data_modelo.DELEGACION_NEGOCIO.apply(MostSimilar, list_ = DELGACIONES)
data_modelo.ESCOLARIDAD = data_modelo.ESCOLARIDAD.apply(MostSimilar, list_ = ESCOLARIDADES)
data_modelo.ESTADO = data_modelo.ESTADO.apply(MostSimilar, list_ = ESTADOS)
data_modelo.ESTADO_NEGOCIO = data_modelo.ESTADO_NEGOCIO.apply(MostSimilar, list_ = ESTADOS)
data_modelo.SUCURSAL = data_modelo.SUCURSAL.apply(MostSimilar, list_ = SUCURSALES)
data_modelo.TIPO_NEGOCIO = data_modelo.TIPO_NEGOCIO.apply(MostSimilar, list_ = TIPOS_NEGOCIO)
data_modelo.TIPO_VIVIENDA = data_modelo.TIPO_VIVIENDA.apply(MostSimilar, list_ = TIPOS_VIVIENDA)
data_modelo.ESTADO_CIVIL = data_modelo.ESTADO_CIVIL.apply(MostSimilar, list_ = ESTADOS_CIVILES)

In [34]:
data_modelo.dtypes

APPLICANT_ID            int64
INGRESO_MENSUAL         int32
GASTOS_FAMILIARES       int32
OTROS_INGRESOS          int32
INGRESO_NETO            int32
DEPENDIENTES            int64
VENTAS                  int64
EDAD                    int32
MONTO_MENSUAL_VENTA     int32
COMPRAS                 int32
DELEGACION             object
DELEGACION_NEGOCIO     object
ESCOLARIDAD            object
ESTADO                 object
ESTADO_CIVIL           object
ESTADO_NEGOCIO         object
SEXO                   object
SUCURSAL               object
TIPO_NEGOCIO           object
TIPO_VIVIENDA          object
dtype: object

In [35]:
data_modelo.MONTO_MENSUAL_VENTA.mean()

17653.23234463277

In [43]:
data_modelo

,APPLICANT_ID,INGRESO_MENSUAL,GASTOS_FAMILIARES,OTROS_INGRESOS,INGRESO_NETO,DEPENDIENTES,VENTAS,EDAD,MONTO_MENSUAL_VENTA,COMPRAS,DELEGACION,DELEGACION_NEGOCIO,ESCOLARIDAD,ESTADO,ESTADO_CIVIL,ESTADO_NEGOCIO,SEXO,SUCURSAL,TIPO_NEGOCIO,TIPO_VIVIENDA
0,145442,8000,800,2500,10800,2,2000,46,8000,900,TENANGO_DEL_AIRE,TENANGO_DEL_AIRE,SECUNDARIA,MEXICO,SOLTERO,MEXICO,FEMENINO,AMECAMECA,CALZADO_REPARACION_Y_VENTA,FAMILIAR
1,145444,12200,0,1000,12200,0,4000,53,16000,1500,TENANGO_DEL_AIRE,TENANGO_DEL_AIRE,TECNICO,MEXICO,CASADO,MEXICO,FEMENINO,AMECAMECA,VENTA_DE_ALIMENTOS,FAMILIAR
2,157169,16000,1300,2100,9600,1,4000,33,16000,1300,NEZAHUALCOYOTL,NEZAHUALCOYOTL,TECNICO,MEXICO,CASADO,MEXICO,FEMENINO,NEZAHUALCOYOTL_2,VENTA_DE_ROPA,FAMILIAR
3,157171,10000,1000,1500,6800,3,2500,24,10000,1200,NEZAHUALCOYOTL,NEZAHUALCOYOTL,BACHILLERATO,MEXICO,UNION LIBRE,MEXICO,FEMENINO,NEZAHUALCOYOTL_2,VENTA_DE_ROPA,FAMILIAR
4,157178,14000,1700,2500,9100,0,3500,53,14000,1800,NEZAHUALCOYOTL,NEZAHUALCOYOTL,SECUNDARIA,MEXICO,SOLTERO,MEXICO,FEMENINO,NEZAHUALCOYOTL_2,JOYERIA/BISUTERIA/FUNDICION,FAMILIAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,167070,11200,1500,2000,5200,1,2800,46,11200,1500,MILPA_ALTA,MILPA_ALTA,SECUNDARIA,DISTRITO_FEDERAL,UNION LIBRE,DISTRITO_FEDERAL,FEMENINO,MILPA_ALTA,VENTA_DE_ROPA,PROPIA
1412,167202,11200,1400,1800,4400,1,2800,33,11200,1500,MILPA_ALTA,MILPA_ALTA,SECUNDARIA,DISTRITO_FEDERAL,UNION LIBRE,DISTRITO_FEDERAL,FEMENINO,MILPA_ALTA,VENTA_DE_ROPA,FAMILIAR
1413,166739,29600,1700,2300,6400,2,7400,25,29600,4000,MILPA_ALTA,MILPA_ALTA,SECUNDARIA,DISTRITO_FEDERAL,UNION LIBRE,DISTRITO_FEDERAL,FEMENINO,MILPA_ALTA,ANIMALES/VETERINARIA,PROPIA
1414,166784,9600,1100,2400,8000,0,2400,56,9600,1400,MILPA_ALTA,MILPA_ALTA,SECUNDARIA,DISTRITO_FEDERAL,UNION LIBRE,DISTRITO_FEDERAL,FEMENINO,MILPA_ALTA,VENTA_DE_ROPA,PROPIA


In [36]:
data_modelo.to_csv("./data/modelo_quash_cleand.csv", index = False)

In [37]:
consulta = data_modelo.to_dict('records')

In [53]:
import json
with open('./data/datos_backtesting_3.json', 'w') as f:
    json.dump(consulta[250:300], f)
with open('./data/datos2.json', 'w') as f:
    json.dump(consulta[400:800], f)
with open('./data/datos3.json', 'w') as f:
    json.dump(consulta[800:], f)

In [59]:
prefix = '{"data": '
suffix = ', "customer": "volana"}'

# Unimos el prefijo, el diccionario serializado como JSON y el sufijo
for i in range(100):
    json_data = prefix + json.dumps(consulta[50*i:50*(i+1)]) + suffix

    file_path = f"./data/datos_finales_hora_si_{i}.json"

    with open(file_path, "w") as json_file:
        json_file.write(json_data)

In [60]:
json_data

'{"data": [], "customer": "volana"}'

In [40]:
#resultado_final = []
DATA = "./data/prueba.json"

In [92]:
resultado_final_hora_si = []

In [63]:
import time
time.sleep(1)

In [129]:
def predict_test(url=URL,data=DATA,endpoint=ENDPOINT):
    with open(data,'r') as _:
        
            global resultado_final_hora_si
            j_data = json.load(_)
            headers = {'content-type': 'application/json',

            'Accept-Charset': 'UTF-8',
            'apikey': APIKEY,
            'x-customer': 'volana'}


            r = requests.post(url + endpoint, data=json.dumps(j_data), headers=headers)
            resultado_final_hora_si.append(r.text)
            print(r, r.text)
        

i = 29
DATA = f"./data/datos_finales_hora_si_{i}.json"
print(DATA)
try:
    fire.Fire(predict_test)
except:
    pass
print(f"Next {i}")
"""
DATA = "./data/datos2.json"
fire.Fire(predict_test)
DATA = "./data/datos3.json"
fire.Fire(predict_test)
"""

./data/datos_finales_hora_si_29.json
<Response [200]> {"prediction":[{"label":"RECHAZADO","approval_score":0.7212736011,"applicant_id":166200},{"label":"RECHAZADO","approval_score":0.884213686,"applicant_id":166201},{"label":"RECHAZADO","approval_score":0.3510470688,"applicant_id":166202},{"label":"APROBADO","approval_score":0.990203619,"applicant_id":166132},{"label":"RECHAZADO","approval_score":0.2759210169,"applicant_id":166260},{"label":"RECHAZADO","approval_score":0.8160589933,"applicant_id":166261},{"label":"RECHAZADO","approval_score":0.781146884,"applicant_id":166262},{"label":"APROBADO","approval_score":0.9408911467,"applicant_id":166263},{"label":"RECHAZADO","approval_score":0.4939048886,"applicant_id":166264},{"label":"RECHAZADO","approval_score":0.606908083,"applicant_id":166741},{"label":"APROBADO","approval_score":0.9548502564,"applicant_id":166893},{"label":"RECHAZADO","approval_score":0.7184106112,"applicant_id":167070},{"label":"RECHAZADO","approval_score":0.5222830772

ERROR: Could not consume arg: -f
Usage: ipykernel_launcher.py -

For detailed information on this command, run:
  ipykernel_launcher.py - --help


Next 29


'\nDATA = "./data/datos2.json"\nfire.Fire(predict_test)\nDATA = "./data/datos3.json"\nfire.Fire(predict_test)\n'

In [88]:
DATA

'./data/datos_finales_hora_si_10.json'

In [141]:
res = pd.DataFrame()
for i in range(30):
    res = pd.concat([res, pd.DataFrame(eval(resultado_final_hora_si[i])['prediction'])])

In [167]:
res.to_csv("./data/resultado_modelo_quash_api.csv", index=False)

In [143]:
data_resultado_quash = data_resultado_quash.merge(res, left_on='id_distribuidor', right_on = 'applicant_id')

In [144]:
data_resultado_quash["perdida_cosecha_esperada_quash"] = data_resultado_quash.approval_score.apply(setcut_prob_quash, grupo_ = False)
data_resultado_quash["grupo_riesgo"] = data_resultado_quash.approval_score.apply(setcut_prob_quash, grupo_ = True)

**Añadiendo capacidad de pago**

In [145]:
ids = "(" + ", ".join(np.array(data_resultado_quash.id_distribuidor, dtype = 'str')) + ")"

In [146]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                               charset='utf8',db = 'volana')
cursor = cnx.cursor()
sql = f"""
select max(id) as 'id_distribuidor', max(capacidad_pago_semanal) as 'capacidad_pago_semanal'
from cc_prospectos_demograficos where id in {ids}
group by id
"""
cursor.execute(sql)
data_capacidad_pago = cursor.fetchall()
data_capacidad_pago = pd.DataFrame(data_capacidad_pago, columns=[i[0] for i in cursor.description])
cnx.close()

In [147]:
data_resultado_quash = data_resultado_quash.merge(data_capacidad_pago, on='id_distribuidor', how = 'left')

In [148]:
data_resultado_quash

,id_distribuidor,tasa_sugerida,grupo_riesgo,ID ASOCIADO,CAPITAL AUTORIZADO,TASA AUTORIZADA,RECUPERADO,NO RECUPERADO,CERRADO,P COSECHA,label,approval_score,applicant_id,perdida_cosecha_esperada_quash,capacidad_pago_semanal
0,145442,147,2,162807,6000,142,6000.000000,0.000000,SI,0.000000,RECHAZADO,0.599554,145442,11.80,2700.0
1,145444,147,5,162806,6000,142,6000.000000,0.000000,SI,0.000000,APROBADO,0.953212,145444,4.06,3050.0
2,157169,147,6,157637,8000,132,8000.000000,0.000000,SI,0.000000,APROBADO,0.989600,157169,1.98,2400.0
3,157171,147,1,157634,11000,132,11000.000000,0.000000,SI,0.000000,RECHAZADO,0.364701,157171,14.76,1700.0
4,157178,147,5,157635,11000,132,11000.000000,0.000000,SI,0.000000,APROBADO,0.944219,157178,4.06,1600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,167070,147,2,167255,9000,142,3398.411536,5601.588464,SI,-0.622399,RECHAZADO,0.718411,167070,11.80,1300.0
1462,167202,0,2,167259,9000,142,3398.411536,5601.588464,SI,-0.622399,RECHAZADO,0.522283,167202,11.80,1100.0
1463,166739,0,2,167256,9000,142,3398.411536,5601.588464,SI,-0.622399,RECHAZADO,0.554235,166739,11.80,1600.0
1464,166784,137,4,167257,9000,142,3398.411536,5601.588464,SI,-0.622399,RECHAZADO,0.884171,166784,6.11,2000.0


In [149]:
from asignarTasa import ajustarTasa

In [169]:
d = 16
t_max = 1.47
resultado = []

for row in data_resultado_quash.iterrows():
    pc = row[1]['perdida_cosecha_esperada_quash']/100
    c  = row[1]['capacidad_pago_semanal']/2
    id_ = int(row[1]['id_distribuidor'])
    prob = float(row[1]['approval_score'])
    grp = row[1]['grupo_riesgo']
    alguno_paso = 0
    for X in range(10_000, 4_000, -1_000):
        estatus, resultado_txt, tasa_minima, tasa_sugerida = ajustarTasa(X, t_max, d, c, pc)
        if estatus:
            resultado.append([id_, resultado_txt, X, tasa_minima, tasa_sugerida, prob, grp])
            alguno_paso = 1
    if alguno_paso == 0:
        _, resultado_txt, _, _ = ajustarTasa(X, t_max, d, c, pc)
        resultado.append([id_, resultado_txt, 0, 0, 0, prob, grp])

data_prospectos_quash = pd.DataFrame(resultado)

data_prospectos_quash.columns = ["id_distribuidor","resultado_txt","monto_autorizado", "tasa_minima","tasa_sugerida", "probabilidad", "grupo_riesgo"]

In [174]:
data_prospectos_quash.grupo_riesgo = data_prospectos_quash.grupo_riesgo.apply(lambda x: "Canasta "+str(x))

In [175]:
data_prospectos_quash.to_csv("./data/data_quash_prospectos_evaluados_formato_aws.csv", index=False)

In [151]:
data_prospectos_quash = data_prospectos_quash.groupby("id_distribuidor").agg({"monto_autorizado": max, 
                                              "tasa_sugerida":max,
                                              "tasa_minima": min,
                                              "probabilidad" : max,
                                              "grupo_riesgo": max})
data_prospectos_quash = data_prospectos_quash.reset_index()

In [168]:
data_prospectos_quash

,id_distribuidor,monto_autorizado,tasa_sugerida,tasa_minima,probabilidad,grupo_riesgo
0,140847,0,0,0,0.549078,2
1,143960,0,0,0,0.504965,2
2,145442,0,0,0,0.599554,2
3,145444,10000,147,115,0.953212,5
4,145969,10000,147,132,0.908972,4
...,...,...,...,...,...,...
1411,166784,10000,147,132,0.884171,4
1412,166882,0,0,0,0.276290,1
1413,166893,10000,147,115,0.954850,5
1414,167070,0,0,0,0.718411,2


In [152]:
data_resultado_quash['tasa_sugerida'] = data_prospectos_quash['tasa_minima']

In [153]:
data_prospectos_quash.tasa_minima.value_counts()

0      853
132    187
142    142
115     89
100     74
137     37
147     17
117      5
127      4
122      3
105      3
110      2
Name: tasa_minima, dtype: int64

# Comparando ambos modelos

In [154]:
data_resultado_quash.head(1)

,id_distribuidor,tasa_sugerida,grupo_riesgo,ID ASOCIADO,CAPITAL AUTORIZADO,TASA AUTORIZADA,RECUPERADO,NO RECUPERADO,CERRADO,P COSECHA,label,approval_score,applicant_id,perdida_cosecha_esperada_quash,capacidad_pago_semanal
0,145442,0.0,2,162807,6000,142,6000.0,0.0,SI,0.0,RECHAZADO,0.599554,145442,11.8,2700.0


In [155]:
data_resultado_quilum.head(1)

,id_distribuidor,tasa_sugerida,grupo_riesgo,ID ASOCIADO,CAPITAL AUTORIZADO,TASA AUTORIZADA,RECUPERADO,NO RECUPERADO,CERRADO,P COSECHA,tasa_minima,probabilidad,perdida_cosecha_esperada_quilum
0,145442,147,Canasta 1,162807,6000,142,6000.0,0.0,SI,0.0,100,0.0,1.63


In [156]:
data_resultado = data_resultado_quilum.copy()

In [157]:
data_resultado.drop(["tasa_sugerida", "grupo_riesgo","tasa_minima", "probabilidad", "perdida_cosecha_esperada_quilum"], axis = 1, 
                    inplace = True)

In [158]:
def shift_mod7(valor):
    shift_map = {1: 7, 2: 6, 3: 5, 4: 4, 5: 3, 6: 2, 7: 1}
    return shift_map.get(valor, valor)

In [159]:
data_resultado['grupo_riesgo_quilum'] = data_resultado_quilum.grupo_riesgo.str.replace("Canasta ","")
data_resultado['grupo_riesgo_quash'] = data_resultado_quash.grupo_riesgo.apply(shift_mod7)

In [160]:
data_resultado['tasa_sugerida_quilum'] = data_resultado_quilum.tasa_minima
data_resultado['tasa_sugerida_quash'] = data_resultado_quash.tasa_sugerida

In [161]:
data_resultado['perdida_cosecha_esperada_quilum'] = data_resultado_quilum.perdida_cosecha_esperada_quilum
data_resultado['perdida_cosecha_esperada_quash'] = data_resultado_quash.perdida_cosecha_esperada_quash

In [162]:
data_resultado['P COSECHA'] = np.abs(data_resultado['P COSECHA'])*100

In [164]:
data_resultado.to_csv("./data/comparativa_modelos.csv", index=True)

## Análisis

In [55]:
data_resultado_analisis = data_resultado.query("not grupo_riesgo_quash.isna()")

**¿Quien se acercó más a la cosecha esperada?**

In [56]:
print("Quash")
np.mean(np.abs(data_resultado_analisis["P COSECHA"] - data_resultado_analisis['perdida_cosecha_esperada_quash']))

Quash


12.440652077225987

In [57]:
print("Quilum")
np.mean(np.abs(data_resultado_analisis["P COSECHA"] - data_resultado_analisis['perdida_cosecha_esperada_quilum']))

Quilum


11.862458119665591

**¿Quién identifica mejor a los buenos pagadores?**

In [58]:
print("Quash")
np.mean(data_resultado_analisis[data_resultado_analisis['P COSECHA']==0]['perdida_cosecha_esperada_quash'])

Quash


7.680805970149255

In [59]:
print("Quilum")
np.mean(data_resultado_analisis[data_resultado_analisis['P COSECHA']==0]['perdida_cosecha_esperada_quilum'])

Quilum


6.558776119402983

**¿Quien dio tasas más altas?**

In [60]:
print("Quash")
np.mean(data_resultado_analisis.query("tasa_sugerida_quash > 0")['tasa_sugerida_quash'])

Quash


131.89067524115757

In [61]:
print("Quilum")
np.mean(data_resultado_analisis.query("tasa_sugerida_quilum>0")['tasa_sugerida_quilum'])

Quilum


116.44813278008299

**Cómo se distribuyen los niveles de riesgo?**

In [62]:
data_resultado_analisis.grupo_riesgo_quash.value_counts().sort_index()

2.0      6
3.0     53
4.0    182
5.0     93
6.0    100
7.0      2
Name: grupo_riesgo_quash, dtype: int64

In [63]:
data_resultado_analisis.grupo_riesgo_quilum.value_counts().sort_index()

1     23
10    44
2     34
3     45
4     36
5     47
6     36
7     52
8     60
9     59
Name: grupo_riesgo_quilum, dtype: int64

## matriz de pérdidas de cosechas Quilum

In [ ]:
num_grupos_riesgo = 7
#cosechas_estimadas = [1.63, 1.84, 1.89, 3.19, 3.76, 5.77, 7.48, 13.17, 15.20, 17.35]
cosechas_estimadas = [1, 1.98, 4.06, 6.11, 7.65, 11.8, 14.76]

In [ ]:
rangos = []
cont = 0
for _ in range(100):
    if cont < 10:
        aum = 1    
    elif cont < 50:
        aum = 5
    else:
        break
        
    rangos.append([cont, int(cont + aum)])
    cont += aum 
    
rangos.append([50, np.inf])

In [ ]:
perdida_cosecha_num_personas = pd.DataFrame(rangos, columns = ["inferior", "superior"])

perdida_cosecha_porcentaje_ideal = pd.DataFrame(rangos, columns = ["inferior", "superior"])

for k in range(1, num_grupos_riesgo + 1):
    perdida_cosecha_num_personas[f"{k}"] = 0 
    perdida_cosecha_porcentaje_ideal[f"{k}"] = 0

In [ ]:
for grupo_riesgo in range(1, num_grupos_riesgo + 1):
    for cos in data_resultado_analisis.query(f"grupo_riesgo_quash == {grupo_riesgo}")["P COSECHA"]:
        for level in perdida_cosecha_num_personas.superior:
            if (cos * 100) < level:
                perdida_cosecha_num_personas.loc[perdida_cosecha_num_personas.superior == level, f"{grupo_riesgo}"] += 1
                break

In [ ]:
divided = perdida_cosecha_num_personas.iloc[:, :].sum(0)
divided[0] = 1
divided[1] = 1

perdida_cosecha_porcentaje = perdida_cosecha_num_personas / divided

In [ ]:
perdida_cosecha_porcentaje

In [ ]:
for grupo_riesgo, nivel in enumerate(cosechas_estimadas):
    for superior in perdida_cosecha_porcentaje_ideal.superior:
        if nivel < superior:
            perdida_cosecha_porcentaje_ideal.loc[perdida_cosecha_porcentaje_ideal.superior == superior, f"{grupo_riesgo+1}"] += 1
            break

In [ ]:
perdida_cosecha_porcentaje_ideal

In [ ]:
np.sum(np.sum(np.abs(perdida_cosecha_porcentaje_ideal.iloc[:,2:] - perdida_cosecha_porcentaje.iloc[:,2:])))

In [ ]:
data_resultado_quash

In [ ]:
data_resultado_analisis